In [18]:
import pandas as pd
import numpy as np

In [2]:
path = '/Users/pranav/nfl_betting_system/data_processing_files/advanced_game_info.csv'
df = pd.read_csv(path)

In [3]:
def normalize(df):
    df = df.replace({True:1, False:0})
    return df
df = normalize(df)

In [4]:
def updateTable(df):
    for idx in range(len(df)):
        if df.loc[idx, 'week'] == 1:
            temp = pd.read_csv('/Users/pranav/nfl_betting_system/team_records/' + str(int(df.loc[idx, 'season'])-1) + '.csv')
            if df.loc[idx, 'team_home'] in set(temp['Tm']):
                df.loc[idx, 'home_win%'] = float(temp.loc[temp['Tm'] == df.loc[idx, 'team_home']]['win%'])
            if df.loc[idx, 'team_away'] in set(temp['Tm']):
                df.loc[idx, 'away_win%'] = float(temp.loc[temp['Tm'] == df.loc[idx, 'team_away']]['win%'])
    return df

In [5]:
df = updateTable(df)

In [6]:
def pythagorean_win_EV(df):
    arr = np.empty((0,2))
    
    for idx, row in df.iterrows():
        
        home_pf = row.home_team_ppg * row.home_team_GP
        home_pa = row.home_team_oppg * row.home_team_GP
        away_pf = row.away_team_ppg * row.away_team_GP
        away_pa = row.away_team_oppg * row.away_team_GP
        
        
        home_exp = (1.5) * np.log(row.home_team_ppg+row.home_team_oppg)
        away_exp = (1.5) * np.log(row.away_team_ppg+row.away_team_oppg)
        
        
        home_py = np.round((np.power(home_pf, home_exp)/(np.power(home_pf, home_exp) + np.power(home_pa, home_exp))), 
                           3)
        away_py = np.round((np.power(away_pf, away_exp)/(np.power(away_pf, away_exp) + np.power(away_pa, away_exp))), 
                           3)
        
        arr = np.append(arr, values = [[home_py, away_py]], axis = 0)
        
    return arr



def addEVs(df):
    temp = pd.DataFrame(pythagorean_win_EV(df), columns=['home_win_exp', 'away_win_exp'])
    
    df = pd.concat([df, temp], axis = 1)
    
    return df

In [7]:
#our pythagorean calculation will result in quite a few 'divide by 0' errors and 'log(0)' ones as well 
#the following lines will ignore these messages
import warnings
warnings.simplefilter('ignore')
df = addEVs(df)

In [8]:
to_drop = ['season', 'week', 'team_home', 'team_away', 'team_favorite_id', 
           'home_team_GP', 'home_team_wins', 'away_team_GP', 'away_team_wins']

df.drop(to_drop, axis=1, inplace=True)
df.dropna(axis=0, inplace=True)
df.index = np.array(range(len(df)))

In [9]:
df.drop(['home_team_id', 'away_team_id', 'winner'], 
        axis=1, 
        inplace=True)

In [10]:
df.drop(['point_diff', 'total_points'], 
        axis=1, 
        inplace=True)

In [11]:
df.sample(15)

,score_home,score_away,spread_favorite,spread_covered,over_under_line,over_under_result,home_team_fav,home_win%,home_team_ppg,home_team_oppg,home_p_diff,away_win%,away_team_ppg,away_team_oppg,away_p_diff,home_win_exp,away_win_exp
3696,38,35,-6.0,0,48.0,1,1,0.500,21.00,18.38,2.62,0.571,25.43,24.71,0.72,0.676,0.542
3311,16,18,-5.5,0,49.0,0,1,0.538,26.62,24.69,1.93,0.538,21.38,20.08,1.30,0.609,0.587
3141,40,32,-2.5,1,49.5,1,1,0.667,27.33,23.00,4.33,1.000,31.67,24.67,7.00,0.734,0.819
3949,30,35,-3.0,0,51.0,1,1,0.500,23.00,21.38,1.62,0.875,27.88,17.50,10.38,0.602,0.935
118,13,16,-2.0,0,39.0,0,1,0.250,17.88,21.38,-3.50,0.500,20.38,21.62,-1.24,0.272,0.418
1174,14,22,-10.0,0,41.0,0,1,0.467,19.67,21.13,-1.46,0.200,16.93,25.07,-8.14,0.402,0.100
3018,24,21,-3.0,0,38.0,1,0,0.400,18.70,20.50,-1.80,0.600,19.80,16.10,3.70,0.376,0.752
4764,28,38,-3.0,1,49.0,1,0,0.571,24.21,19.93,4.28,0.786,27.00,21.64,5.36,0.751,0.784
4158,21,13,-7.0,1,48.5,0,1,0.714,27.86,23.71,4.15,0.429,19.57,18.71,0.86,0.722,0.561
859,24,14,-10.5,0,47.5,0,1,0.600,26.90,23.30,3.60,0.300,16.60,24.20,-7.60,0.699,0.109


In [19]:
df.to_json('/Users/pranav/nfl_betting_system/final_data.json')
df.to_csv('/Users/pranav/nfl_betting_system/final_data.csv')